# STEM JOBS

- O objetivo desse trabalho foi criar um df o qual conteria todos os indivíduos que trabalharam nos CNAE (segmentos econômicos) em que a Odebrecht operava em coortes diferentes (2012 a 2016)

- Esses dados seriam posteriormente utilizados em uma análise econométrica para estimar o efeito da operação Lava Jato na criação de "manager/director spin-offs", ou seja, na criação de empresas novas por ex-CEOs da Odebrecht

- Todos os dados que identificam indivíduos ou informações sigilosas foram removidos deste script em respeito as normas da LGPD

# Configurando o ambiente

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import basedosdados as bd
import pyodbc
import re
import time

sns.set(rc={'figure.figsize':(11.7,8.27)})
sns.set_theme(style="darkgrid")

# Criando lista Odebrecht e Camargo

- A lista de empresas da Odebrecht e da Camargo Correa foi recolhida da Revista Valor Grandes Grupos

In [3]:
ode_cam = pd.read_csv("Dados/Input/220414_empresas_valor_grandes_grupos.csv", sep = ";", dtype ='str',  encoding='latin-1')

In [4]:
ode_cam.head()

,empresa,grupo,cnpj,ano
0,Nordeste Quimica S.A. Norquisa,odebrecht,15.659.535/0001-46,2015
1,Odebrecht Serviços e Participações S.A.,odebrecht,10.904.193/0001-69,2015
2,ODBINV S.A.,odebrecht,15.105.588/0001-15,2015
3,Odebrecht Participações e Engenharia S.A.,odebrecht,17.851.495/0001-65,2015
4,Odebrecht Realizações Imobiliárias e Participa...,odebrecht,06.206.132/0001-50,2015


In [5]:
ode_cam["cnpj_limpo"] = ode_cam["cnpj"].apply(lambda x: re.sub('[^0-9]', '', str(x)[:10]))

In [6]:
ode_cam.head(10)

,empresa,grupo,cnpj,ano,cnpj_limpo
0,Nordeste Quimica S.A. Norquisa,odebrecht,15.659.535/0001-46,2015,15659535
1,Odebrecht Serviços e Participações S.A.,odebrecht,10.904.193/0001-69,2015,10904193
2,ODBINV S.A.,odebrecht,15.105.588/0001-15,2015,15105588
3,Odebrecht Participações e Engenharia S.A.,odebrecht,17.851.495/0001-65,2015,17851495
4,Odebrecht Realizações Imobiliárias e Participa...,odebrecht,06.206.132/0001-50,2015,06206132
5,Odebrecht Agroindustrial Investimentos S.A,odebrecht,NaN,2015,
6,Odebrecht Energia S.A,odebrecht,13.079.757/0002-45,2015,13079757
7,Braskem S.A.,odebrecht,42.150.391/0001-70,2015,42150391
8,Enseada indústria Naval Participações S.A.,odebrecht,12.243.301/0001-25,2015,12243301
9,Odebrecht S.A.,odebrecht,05.144.757/0005-04,2015,05144757


In [7]:
len(ode_cam["cnpj_limpo"])

468

In [8]:
ode_cam = ode_cam.dropna()

In [9]:
len(ode_cam)

292

In [10]:
ode_cam.to_csv("Dados/Output/220413_cnpjs_odebrecht_camargo.csv", sep = ";", index = False, encoding = "utf-8-sig" )

In [11]:
cnpjs = tuple(ode_cam["cnpj_limpo"])

In [12]:
cnpjs_odebrecht = tuple(ode_cam.query("grupo == 'odebrecht'").cnpj_limpo)

# Baixando dados da RAIS

In [13]:
server = 'beirute-01'

database = 'RAIS'

port= '***'

username = '****'

password = '****'

driver= '{SQL Server}'

cnxn = pyodbc.connect('DRIVER='+driver+';SERVER='+server+';PORT='+port+';DATABASE='+database+';UID='+username+';PWD='+ password)

cursor = cnxn.cursor()

# Encontrando CNAES Odebrecht

In [14]:
estados = ["SP","RJ","BA","MG","PR"] # estados de interesse

In [15]:
cnaes = []
df = pd.DataFrame()

In [16]:
def cnae(ano,estado):
    global df, cnaes
    consulta_1 = """ SELECT
                 a.CPF,
                 a.[Nome Trabalhador],
                 a.[Município],
                 a.[CNAE 95 Classe],
                 a.[Sexo Trabalhador],
                 a.[Raça Cor],
                 a.[Idade],
                 a.[CNPJ Raiz],
                 a.[CBO Ocupação 2002],
                 a.[CNAE 2 0 Classe] as cnae,
                 a.[Vl Salário Contratual],
                 a.[Vl Remun Média (SM)],
                 a.[Vl Remun Média Nom],
                 a.[Vl Remun Dezembro Nom],
                 a.[Tipo Vínculo],
                 a.[Motivo Desligamento],
                 a.[Mês Desligamento],
                 a.[Escolaridade após 2005],
                 a.[Natureza Jurídica],
                 a.[Data Admissão Declarada],
                 a.[Tempo Emprego],
                 a.[CNAE 2 0 Subclasse] as cnae_sub,
                 a.[Vínculo Ativo 31 12] as vinc_ativ3112
                 FROM dbo.{}{}ID a
                 WHERE a.[Escolaridade após 2005] >= 7
                 AND [CNPJ Raiz] IN {}
                """.format(estado,ano,cnpjs_odebrecht)

    df_provisorio = pd.read_sql(consulta_1,  cnxn)

    print(df_provisorio["vinc_ativ3112"].unique())
    
    cnaes = list(dict.fromkeys(cnaes + list(df_provisorio.cnae.unique())))

In [17]:
start_time = time.time()

for x in estados:
    cnae(2015,x)

print("My program took", time.time() - start_time, "to run")

[1 0]
[1 0]
[1 0]
[0 1]
[1 0]
My program took 104.31728458404541 to run


In [18]:
lista = []
for i in cnaes:
    lista.append(i[0:5])

In [19]:
cnaes = tuple(lista)

In [20]:
cnaes

('42120',
 '20215',
 '72100',
 '36006',
 '71120',
 '64620',
 '70204',
 '19314',
 '20312',
 '52311',
 '52214',
 '37011',
 '41204',
 '42138',
 '66223',
 '38211',
 '42227',
 '42928',
 '73122',
 '52401',
 '49124',
 '41107',
 '46931',
 '09106',
 '33171',
 '52508',
 '10651',
 '38220',
 '01351',
 '37029',
 '20118',
 '25993',
 '46893')

In [21]:
len(cnaes)

33

## Criando Painel 2012

In [22]:
estados = ["SP","RJ","BA","MG","PR"] # estados de interesse
anos = [2012,2013,2014] # anos do painel

In [23]:
cpfs = []
df = pd.DataFrame()

In [24]:
def raispainel(ano,estado):
    global df, cpfs
    consulta_1 = """ SELECT
                 a.CPF,
                 a.[Nome Trabalhador],
                 a.[Município],
                 a.[CNAE 95 Classe],
                 a.[Sexo Trabalhador],
                 a.[Raça Cor],
                 a.[Idade],
                 a.[CNPJ Raiz],
                 a.[CBO Ocupação 2002] as cbo2002,
                 a.[CBO 94 Ocupação] as cbo_94,
                 a.[CNAE 2 0 Classe] as cnae,
                 a.[Vl Salário Contratual],
                 a.[Vl Remun Média (SM)],
                 a.[Vl Remun Média Nom],
                 a.[Vl Remun Dezembro Nom],
                 a.[Tipo Vínculo],
                 a.[Motivo Desligamento],
                 a.[Mês Desligamento],
                 a.[Escolaridade após 2005],
                 a.[Natureza Jurídica],
                 a.[Data Admissão Declarada],
                 a.[Tempo Emprego],
                 a.[CNAE 2 0 Subclasse] as cnae_sub,
                 a.[Vínculo Ativo 31 12] as vinc_ativ3112
                 FROM dbo.{}{}ID a
                 WHERE a.[Escolaridade após 2005] >= 7
                 AND a.[Vínculo Ativo 31 12] != '1'
                 AND a.[Vínculo Ativo 31 12] != ' 1'
                 AND a.[Vínculo Ativo 31 12] != 1
                 AND a.[CNAE 2 0 Classe] IN {}
                """.format(estado,ano,cnaes)

    df_provisorio = pd.read_sql(consulta_1,  cnxn)

    df_provisorio["Ano"] = ano
    df_provisorio["Estado"] = estado
    df_provisorio["painel"] = anos[0]
    
    
    df_provisorio['CPF'] = df_provisorio['CPF'].str.strip()

    
    df_provisorio.loc[(df_provisorio['vinc_ativ3112'] == ' 1') |
                      (df_provisorio['vinc_ativ3112'] == '1')  |
                      (df_provisorio['vinc_ativ3112'] == 'True'), 'vinc_ativ3112'] = 1
    
    df_provisorio.loc[(df_provisorio['vinc_ativ3112'] == ' 0') |
                      (df_provisorio['vinc_ativ3112'] == '0')  |
                      (df_provisorio['vinc_ativ3112'] == 'False'),'vinc_ativ3112'] = 0
    
    df = pd.concat([df, df_provisorio])


    cpfs = list(dict.fromkeys(cpfs + list(df_provisorio.query("vinc_ativ3112 != 1").CPF.unique())))


    print(f"foi criado um df de tamanho {len(df)} e uma lista chamada cpf de tamanho {len(cpfs)}")

In [25]:

start_time = time.time()

for x in estados:
    raispainel(anos[0],x)

print("My program took", time.time() - start_time, "to run")

foi criado um df de tamanho 284969 e uma lista chamada cpf de tamanho 273057
foi criado um df de tamanho 373133 e uma lista chamada cpf de tamanho 355472
foi criado um df de tamanho 445446 e uma lista chamada cpf de tamanho 421687
foi criado um df de tamanho 554742 e uma lista chamada cpf de tamanho 523831
foi criado um df de tamanho 622825 e uma lista chamada cpf de tamanho 587014
My program took 37.035183906555176 to run


In [26]:
df["vinc_ativ3112"].unique()

array([0], dtype=int64)

In [27]:
def raispainel2(ano,estado):
    global df
    consulta_1 = """ SELECT
                 a.CPF,
                 a.[Nome Trabalhador],
                 a.[Município],
                 a.[CNAE 95 Classe],
                 a.[Sexo Trabalhador],
                 a.[Raça Cor],
                 a.[Idade],
                 a.[CNPJ Raiz],
                 a.[CBO Ocupação 2002] as cbo2002,
                 a.[CBO 94 Ocupação] as cbo_94,
                 a.[CNAE 2 0 Classe] as cnae,
                 a.[Vl Salário Contratual],
                 a.[Vl Remun Média (SM)],
                 a.[Vl Remun Média Nom],
                 a.[Vl Remun Dezembro Nom],
                 a.[Tipo Vínculo],
                 a.[Motivo Desligamento],
                 a.[Mês Desligamento],
                 a.[Escolaridade após 2005],
                 a.[Natureza Jurídica],
                 a.[Data Admissão Declarada],
                 a.[Tempo Emprego],
                 a.[CNAE 2 0 Subclasse] as cnae_sub,
                 a.[Vínculo Ativo 31 12] as vinc_ativ3112
                 FROM dbo.{}{}ID a
                 WHERE a.[Vínculo Ativo 31 12] != '0'
                 AND a.[Vínculo Ativo 31 12] != ' 0'
                 AND a.[Vínculo Ativo 31 12] != 0
                 AND a.[Vínculo Ativo 31 12] != ''
                 AND a.[Escolaridade após 2005] >= 7
                """.format(estado,ano)

    df_provisorio = pd.read_sql(consulta_1,  cnxn)
    
    df_provisorio['CPF'] = df_provisorio['CPF'].str.strip()
    
    df_provisorio = df_provisorio.query("CPF in @cpfs")
    
    df_provisorio["Ano"] = ano
    df_provisorio["Estado"] = estado
    df_provisorio["painel"] = anos[0]
    
    df = pd.concat([df, df_provisorio])


    print(f"foi criado um df de tamanho {len(df)}")

In [28]:

start_time = time.time()

for x in estados:
    raispainel2(anos[1],x)
    raispainel2(anos[2],x)

print("My program took", time.time() - start_time, "to run")

foi criado um df de tamanho 738623
foi criado um df de tamanho 851801
foi criado um df de tamanho 893757
foi criado um df de tamanho 935369
foi criado um df de tamanho 956030
foi criado um df de tamanho 976370
foi criado um df de tamanho 1015348
foi criado um df de tamanho 1053725
foi criado um df de tamanho 1073290
foi criado um df de tamanho 1092875
My program took 997.9832656383514 to run


In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1092875 entries, 0 to 2142962
Data columns (total 27 columns):
 #   Column                   Non-Null Count    Dtype  
---  ------                   --------------    -----  
 0   CPF                      1092875 non-null  object 
 1   Nome Trabalhador         1092875 non-null  object 
 2   Município                1092875 non-null  int64  
 3   CNAE 95 Classe           1092875 non-null  int64  
 4   Sexo Trabalhador         1092875 non-null  int64  
 5   Raça Cor                 1092875 non-null  int64  
 6   Idade                    1092875 non-null  int64  
 7   CNPJ Raiz                1092875 non-null  object 
 8   cbo2002                  1092875 non-null  object 
 9   cbo_94                   1092875 non-null  object 
 10  cnae                     1092875 non-null  object 
 11  Vl Salário Contratual    1092875 non-null  float64
 12  Vl Remun Média (SM)      1092875 non-null  float64
 13  Vl Remun Média Nom       1092875 non-null 

In [30]:
df["vinc_ativ3112"].unique()

array([0, 1], dtype=int64)

# Criando painel 2013

In [31]:
estados = ["SP","RJ","BA","MG","PR"] # estados de interesse
anos = [2013,2014,2015] # anos do painel

cpfs = []

In [32]:
start_time = time.time()

for x in estados:
    raispainel(anos[0],x)

print("My program took", time.time() - start_time, "to run")

foi criado um df de tamanho 1371752 e uma lista chamada cpf de tamanho 266878
foi criado um df de tamanho 1475008 e uma lista chamada cpf de tamanho 365187
foi criado um df de tamanho 1549520 e uma lista chamada cpf de tamanho 432992
foi criado um df de tamanho 1662448 e uma lista chamada cpf de tamanho 538737
foi criado um df de tamanho 1726179 e uma lista chamada cpf de tamanho 598041
My program took 34.96587371826172 to run


In [33]:
start_time = time.time()

for x in estados:
    raispainel2(anos[1],x)
    raispainel2(anos[2],x)

print("My program took", time.time() - start_time, "to run")

foi criado um df de tamanho 1835304
foi criado um df de tamanho 1936886
foi criado um df de tamanho 1980697
foi criado um df de tamanho 2019353
foi criado um df de tamanho 2040979
foi criado um df de tamanho 2060740
foi criado um df de tamanho 2097066
foi criado um df de tamanho 2129613
foi criado um df de tamanho 2149427
foi criado um df de tamanho 2168808
My program took 1342.6583161354065 to run


In [35]:
df.painel.unique()

array([2012, 2013], dtype=int64)

# Criando painel 2014

In [36]:
estados = ["SP","RJ","BA","MG","PR"] # estados de interesse
anos = [2014,2015,2016] # anos do painel

cpfs = []

In [37]:
start_time = time.time()

for x in estados:
    raispainel(anos[0],x)

print("My program took", time.time() - start_time, "to run")

foi criado um df de tamanho 2461796 e uma lista chamada cpf de tamanho 278440
foi criado um df de tamanho 2569095 e uma lista chamada cpf de tamanho 379963
foi criado um df de tamanho 2637314 e uma lista chamada cpf de tamanho 442565
foi criado um df de tamanho 2766244 e uma lista chamada cpf de tamanho 561916
foi criado um df de tamanho 2831618 e uma lista chamada cpf de tamanho 621737
My program took 62.76587128639221 to run


In [38]:
start_time = time.time()

for x in estados:
    raispainel2(anos[1],x)
    raispainel2(anos[2],x)

print("My program took", time.time() - start_time, "to run")

foi criado um df de tamanho 2934975
foi criado um df de tamanho 3032816
foi criado um df de tamanho 3073919
foi criado um df de tamanho 3110609
foi criado um df de tamanho 3129380
foi criado um df de tamanho 3146871
foi criado um df de tamanho 3179394
foi criado um df de tamanho 3210023
foi criado um df de tamanho 3230066
foi criado um df de tamanho 3248366
My program took 2923.2159395217896 to run


In [40]:
df.painel.unique()

array([2012, 2013, 2014], dtype=int64)

# Criando painel 2015

In [41]:
estados = ["SP","RJ","BA","MG","PR"] # estados de interesse
anos = [2015,2016,2017] # anos do painel

cpfs = []

In [42]:
start_time = time.time()

for x in estados:
    raispainel(anos[0],x)

print("My program took", time.time() - start_time, "to run")

foi criado um df de tamanho 3509726 e uma lista chamada cpf de tamanho 247693
foi criado um df de tamanho 3618546 e uma lista chamada cpf de tamanho 350867
foi criado um df de tamanho 3685274 e uma lista chamada cpf de tamanho 412012
foi criado um df de tamanho 3803427 e uma lista chamada cpf de tamanho 520004
foi criado um df de tamanho 3867939 e uma lista chamada cpf de tamanho 577847
My program took 87.46789693832397 to run


In [43]:
start_time = time.time()

for x in estados:
    raispainel2(anos[1],x)
    raispainel2(anos[2],x)

print("My program took", time.time() - start_time, "to run")

foi criado um df de tamanho 3956405
foi criado um df de tamanho 4047273
foi criado um df de tamanho 4081109
foi criado um df de tamanho 4116188
foi criado um df de tamanho 4132559
foi criado um df de tamanho 4150269
foi criado um df de tamanho 4178518
foi criado um df de tamanho 4209044
foi criado um df de tamanho 4227154
foi criado um df de tamanho 4245036
My program took 4745.877126216888 to run


In [45]:
df.painel.unique()

array([2012, 2013, 2014, 2015], dtype=int64)

# Criando painel 2016

In [46]:
estados = ["SP","RJ","BA","MG","PR"] # estados de interesse
anos = [2016,2017,2018] # anos do painel

cpfs = []

In [47]:
start_time = time.time()

for x in estados:
    raispainel(anos[0],x)

print("My program took", time.time() - start_time, "to run")

foi criado um df de tamanho 4483200 e uma lista chamada cpf de tamanho 223772
foi criado um df de tamanho 4574448 e uma lista chamada cpf de tamanho 311132
foi criado um df de tamanho 4630729 e uma lista chamada cpf de tamanho 362269
foi criado um df de tamanho 4733552 e uma lista chamada cpf de tamanho 454976
foi criado um df de tamanho 4791345 e uma lista chamada cpf de tamanho 505990
My program took 94.98253560066223 to run


In [48]:
start_time = time.time()

for x in estados:
    raispainel2(anos[1],x)
    raispainel2(anos[2],x)

print("My program took", time.time() - start_time, "to run")

foi criado um df de tamanho 4877038
foi criado um df de tamanho 4966582
foi criado um df de tamanho 4996837
foi criado um df de tamanho 5029330
foi criado um df de tamanho 5044588
foi criado um df de tamanho 5060433
foi criado um df de tamanho 5088408
foi criado um df de tamanho 5118331
foi criado um df de tamanho 5135314
foi criado um df de tamanho 5152105
My program took 4677.123672008514 to run


In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5152105 entries, 0 to 2271248
Data columns (total 27 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   CPF                      object 
 1   Nome Trabalhador         object 
 2   Município                object 
 3   CNAE 95 Classe           object 
 4   Sexo Trabalhador         object 
 5   Raça Cor                 object 
 6   Idade                    int64  
 7   CNPJ Raiz                object 
 8   cbo2002                  object 
 9   cbo_94                   object 
 10  cnae                     object 
 11  Vl Salário Contratual    float64
 12  Vl Remun Média (SM)      float64
 13  Vl Remun Média Nom       float64
 14  Vl Remun Dezembro Nom    float64
 15  Tipo Vínculo             object 
 16  Motivo Desligamento      object 
 17  Mês Desligamento         object 
 18  Escolaridade após 2005   object 
 19  Natureza Jurídica        object 
 20  Data Admissão Declarada  object 
 21  Tempo Em

In [50]:
df.painel.unique()

array([2012, 2013, 2014, 2015, 2016], dtype=int64)

## Criando variável "cnae_odebrecht_2015"

In [51]:
df["cnae"] = df["cnae"].apply(lambda x: str(x)[0:5])

In [52]:
df["cnae"].head()

0    46931
1    41204
2    41204
3    66223
4    41204
Name: cnae, dtype: object

In [53]:
df["cnae_odebrecht_2015"] = df["cnae"].apply(lambda x: 1 if x in cnaes else 0)

## Criando variável "stem_job"

In [54]:
cbos_stem = "1223 1236 1237 1413 1425 1426 2011 2012 2021 2030 2031 2032 2033 2034 2111 2112 2122 2123 2124 2131 2132 2133 2134 2140 2142 2143 2144 2145 2146 2147 2148 2149 2211 2212 2221 2222 2341 2342 2343 2344 2513 3001 3003 3011 3012 3111 3112 3115 3121 3122 3123 3161 3171 3172 3180 3181 3182 3183 3185 3186 3187 3201 3212 3253 3951"

- A ideia aqui é definir quais empregos são considerados como de Ciência, Tecnologia, Engenharia e Matemática
- fonte: (Machado et all 2021) 

In [55]:
cbos_stem = tuple(cbos_stem.split(" "))

In [56]:
df["stem_job"] = df["cbo2002"].apply(lambda x: 1 if x[0:4] in cbos_stem else 0)

## Criando variável "stem_job_2015"

In [57]:
cpfs_stem_2015 = df[(df["stem_job"] == 1) & (df["Ano"] == 2015)].CPF.unique()

In [58]:
df["stem_job_2015"] = df["CPF"].apply(lambda x: 1 if x in cpfs_stem_2015 else 0)

In [59]:
df.query("stem_job_2015 == 1").Ano.unique()

array([2012, 2013, 2014, 2015, 2016, 2017, 2018], dtype=int64)

In [60]:
len(df.query("stem_job_2015 == 1 & Ano != 2015"))

179067

## Criando variável "odebrecht"

In [61]:
cnpjs_odebrecht = tuple(ode_cam.query("grupo == 'odebrecht'").cnpj_limpo)

In [62]:
df["odebrecht"] = df["CNPJ Raiz"].apply(lambda x: 1 if x in cnpjs_odebrecht else 0)

## Criando variável "odebrecht_2015"

In [64]:
cpfs_odebrecht = df[(df["odebrecht"] == 1) & (df["Ano"] == 2015)].CPF.unique()

In [65]:
df["odebrecht_2015"] = df["CPF"].apply(lambda x: 1 if x in cpfs_odebrecht else 0)

In [66]:
df.query("odebrecht_2015 == 1").Ano.unique()

array([2012, 2013, 2014, 2015, 2016, 2017, 2018], dtype=int64)

In [67]:
len(df.query("odebrecht_2015 == 1 & Ano == 2015"))

19605

## Saving

In [68]:
df.loc[(df['vinc_ativ3112'] == ' 1') |
       (df['vinc_ativ3112'] == '1')  |
       (df['vinc_ativ3112'] == 'True'), 'vinc_ativ3112'] = 1

In [69]:
df["vinc_ativ3112"].unique()

array([0, 1], dtype=object)

In [70]:
df.to_csv("Dados/Output/220517_all_panel_cnae.csv", sep = ";", index = False, encoding = "utf-8-sig")

# Abrindo e Limpando a Base

In [71]:
df = pd.read_csv("Dados/Output/220517_all_panel_cnae.csv", sep = ";", dtype = {'CPF': str, 'CNPJ Raiz': str, 'CBO Ocupação 2002': str, 'Data Admissão Declarada': str, 'cnae': str, "Vínculo Ativo 31 12": str})

In [72]:
len(df.CPF.unique())

2440241

In [73]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5152105 entries, 0 to 5152104
Data columns (total 32 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   CPF                      object 
 1   Nome Trabalhador         object 
 2   Município                int64  
 3   CNAE 95 Classe           int64  
 4   Sexo Trabalhador         int64  
 5   Raça Cor                 int64  
 6   Idade                    int64  
 7   CNPJ Raiz                object 
 8   cbo2002                  object 
 9   cbo_94                   object 
 10  cnae                     object 
 11  Vl Salário Contratual    float64
 12  Vl Remun Média (SM)      float64
 13  Vl Remun Média Nom       float64
 14  Vl Remun Dezembro Nom    float64
 15  Tipo Vínculo             int64  
 16  Motivo Desligamento      int64  
 17  Mês Desligamento         int64  
 18  Escolaridade após 2005   int64  
 19  Natureza Jurídica        int64  
 20  Data Admissão Declarada  object 
 21  Tempo Em

In [74]:
len(df[df["vinc_ativ3112"] == 1])

1951016

In [75]:
len(df[df["vinc_ativ3112"] == True])

1951016

In [76]:
df["vinc_ativ3112"].unique()

array([0, 1, '1', 'True', 'False'], dtype=object)

In [77]:
df.loc[(df['vinc_ativ3112'] == ' 1') |
       (df['vinc_ativ3112'] == '1')  |
       (df['vinc_ativ3112'] == 'True'), 'vinc_ativ3112'] = 1

In [78]:
df.loc[(df['vinc_ativ3112'] == ' 0') |
       (df['vinc_ativ3112'] == '0')  |
       (df['vinc_ativ3112'] == 'False'), 'vinc_ativ3112'] = 0

In [79]:
df["vinc_ativ3112"].unique()

array([0, 1], dtype=object)

In [80]:
df['cbo2002'] = df['cbo2002'].str.replace('\W', '')

In [81]:
df.rename(columns={"painel": "coorte"}, inplace = True)

In [82]:
df["cnae"] = df["cnae"].apply(lambda x: str(x)[0:5])

# Criando Grupos Econômicos

## Criando Dummy "Odebrecht"

In [84]:
cnpjs_odebrecht = tuple(ode_cam.query("grupo == 'odebrecht'").cnpj_limpo)

In [85]:
df["Odebrecht"] = df["CNPJ Raiz"].apply(lambda x: 1 if x in cnpjs_odebrecht else 0)

## Criar Dummy Camargo Correa

In [88]:
cnpjs_camargo = tuple(ode_cam.query("grupo == 'camargo'").cnpj_limpo)

In [89]:
df["Camargo"] = df["CNPJ Raiz"].apply(lambda x: 1 if x in cnpjs_camargo else 0)

# Gerando variável "empreendedor"

# Gerando variável "CEO_LT"

- Aqui vou gerar a principal variável de CEO, com base em (Hirakawa et al, 2010)

## Pegar CBOs de "directors" ou "managers"

In [91]:
cbos_1994 = pd.read_csv("Dados/Input/cbo-isco-conc.csv", sep = ",", dtype ='str',  encoding='latin-1')

- Tabela de Marc Muendler disponível em: https://econweb.ucsd.edu/muendler/html/brazil.html#brazoccu
    - Metodologia: (Muendler, 2004, p.16)

In [92]:
cbos_1994 = cbos_1994.dropna()

In [93]:
cbos_1994_managers = cbos_1994.loc[cbos_1994.cbotrans.str.contains('Manager', case = False)]

In [94]:
cbos_1994_directors = cbos_1994.loc[cbos_1994.cbotrans.str.contains('Director', case = False)]

In [95]:
len(cbos_1994_managers)

54

In [96]:
len(cbos_1994_directors)

46

In [97]:
cbos_1994_managers['CBO94'] = cbos_1994_managers['cboid'].str.replace('\W', '')

In [98]:
cbos_1994_directors['CBO94'] = cbos_1994_directors['cboid'].str.replace('\W', '')

## Conversao CBOs 94 para 2002

- Com base em Ricardo Dahis: https://github.com/rdahis/clean_RAIS/tree/master/RAIS/extra/CBO/conversao%2094-02

In [99]:
cbos_2002_managers_directors = pd.read_csv("Dados\Input\CBO94 - CBO2002 - Conversao com 90.csv", sep = ";", dtype ='str')

In [100]:
cbos_2002_managers_directors.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3897 entries, 0 to 3896
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   CBO94    3897 non-null   object
 1   CBO2002  3897 non-null   object
dtypes: object(2)
memory usage: 61.0+ KB


In [101]:
cbos_2002_managers_directors.head()

,CBO94,CBO2002
0,X4010,020105
1,X4020,020205
2,X4030,021105
3,X4040,021210
4,X5010,010110


## Juntando bases

In [103]:
cbos_2002_managers = cbos_2002_managers_directors.merge(cbos_1994_managers, on='CBO94', how='inner')

In [104]:
len(cbos_2002_managers)

170

In [105]:
cbos_2002_directors = cbos_2002_managers_directors.merge(cbos_1994_directors, on='CBO94', how='inner')

In [106]:
len(cbos_2002_directors)

80

In [107]:
df["cbo_2002_manager"] = df["cbo2002"].apply(lambda x: 1 if x in list(cbos_2002_managers.CBO2002.unique()) else 0)

In [108]:
df.query("cbo_2002_manager == 1").Ano.unique()

array([2012, 2013, 2014, 2015, 2016, 2017, 2018], dtype=int64)

In [109]:
len(df.query("cbo_2002_manager == 1").CPF.unique()) / len(df.CPF.unique())

0.1712621827106421

In [112]:
df["cbo_2002_director"] = df["cbo2002"].apply(lambda x: 1 if x in list(cbos_2002_directors.CBO2002.unique()) else 0)

In [113]:
df.query("cbo_2002_director == 1").Ano.unique()

array([2012, 2013, 2014, 2015, 2016, 2017, 2018], dtype=int64)

In [114]:
len(df.query("cbo_2002_director == 1").CPF.unique()) / len(df.CPF.unique())

0.042908466827661695

# Salvando

In [117]:
df.to_csv("Dados/Output/220517_all_panel_cnae_ceo.csv", sep = ";", index = False, encoding = "utf-8-sig")

# Criando Tabela Sum Statiscs

In [131]:
df.rename(columns={"Escolaridade após 2005": "estudo", "CNPJ Raiz": "cnpj"}, inplace = True)

In [137]:
def sumstats(ano):
    global tabela
    cpfs_odebrecht = df[(df["Odebrecht"] == 1) & (df["Ano"] == ano)].CPF.unique()
    
    cpfs_camargo = df[(df["Camargo"] == 1) & (df["Ano"] == ano)].CPF.unique()
    
    n_entrepreneur_odebrecht = len(df.query("CPF in @cpfs_odebrecht")[
                                                                      (df["coorte"] == ano) &
                                                                      (df["cbo_2002_director"] == 1) &
                                                                      (df["Odebrecht"] == 0) &
                                                                      (df["Camargo"] == 0)].CPF.unique())
    n_entrepreneur_camargo = len(df.query("CPF in @cpfs_camargo")[
                                                                  (df["coorte"] == ano) &
                                                                  (df["cbo_2002_director"] == 1) &
                                                                  (df["Odebrecht"] == 0) &
                                                                  (df["Camargo"] == 0)].CPF.unique())
    
    n_bluecollar_odebrecht = len(df.query("CPF in @cpfs_odebrecht")[(df["cbo_2002_director"] == 0) &
                                                                    (df["coorte"] == ano) &
                                                                    (df["Odebrecht"] == 0)].CPF.unique())

    n_bluecollar_camargo = len(df.query("CPF in @cpfs_camargo")[(df["cbo_2002_director"] == 0) &
                                                                (df["coorte"] == ano) &
                                                                (df["Camargo"] == 0)].CPF.unique())
    
    perc_entrepreneur_odebrecht = round((n_entrepreneur_odebrecht / (n_entrepreneur_odebrecht + n_bluecollar_odebrecht))*100, 2) 
    perc_entrepreneur_camargo = round((n_entrepreneur_camargo / (n_entrepreneur_camargo + n_bluecollar_camargo))*100, 2)
    perc_bluecollar_odebrecht = 100 - perc_entrepreneur_odebrecht
    perc_bluecollar_camargo = 100 - perc_entrepreneur_camargo
    
    study_entrepreneur_odebrecht = round(df.query("CPF in @cpfs_odebrecht")[
                                                                            (df["coorte"] == ano) &
                                                                            (df["cbo_2002_director"] == 1) &
                                                                            (df["Odebrecht"] == 0) &
                                                                            (df["Camargo"] == 0)].estudo.median(),2)
    
    study_entrepreneur_camargo = round(df.query("CPF in @cpfs_camargo")[
                                                                        (df["coorte"] == ano) &
                                                                        (df["cbo_2002_director"] == 1) &
                                                                        (df["Odebrecht"] == 0) &
                                                                        (df["Camargo"] == 0)].estudo.median(),2)
    
    study_bluecollar_odebrecht = round(df.query("CPF in @cpfs_odebrecht")[(df["cbo_2002_director"] == 0) &
                                                                          (df["coorte"] == ano) &
                                                                          (df["Odebrecht"] == 0)].estudo.median(),2)
    
    study_bluecollar_camargo = round(df.query("CPF in @cpfs_camargo")[(df["cbo_2002_director"] == 0) &
                                                                      (df["coorte"] == ano) &
                                                                      (df["Camargo"] == 0)].estudo.median(),2)
    
    age_entrepreneur_odebrecht = round(df.query("CPF in @cpfs_odebrecht")[
                                                                          (df["coorte"] == ano) &
                                                                          (df["cbo_2002_director"] == 1) &
                                                                          (df["Odebrecht"] == 0) &
                                                                          (df["Camargo"] == 0)].Idade.mean(),2)
     
    age_entrepreneur_camargo = round(df.query("CPF in @cpfs_camargo")[
                                                                      (df["coorte"] == ano) &
                                                                      (df["cbo_2002_director"] == 1) &
                                                                      (df["Odebrecht"] == 0) &
                                                                      (df["Camargo"] == 0)].Idade.mean(),2)
    
    age_bluecollar_odebrecht = round(df.query("CPF in @cpfs_odebrecht")[(df["cbo_2002_director"] == 0) &
                                                                        (df["coorte"] == ano) &
                                                                        (df["Odebrecht"] == 0)].Idade.mean(),2)
    
    age_bluecollar_camargo = round(df.query("CPF in @cpfs_camargo")[(df["cbo_2002_director"] == 0) &
                                                                    (df["coorte"] == ano) &
                                                                    (df["Camargo"] == 0)].Idade.mean(),2)
    
    tenure_entrepreneur_odebrecht_cpfs = df.query("CPF in @cpfs_odebrecht")[
                                                                            (df["coorte"] == ano) &
                                                                            (df["cbo_2002_director"] == 1) &
                                                                            (df["Odebrecht"] == 0) &
                                                                            (df["Camargo"] == 0)].CPF.unique()
    
    tenure_entrepreneur_odebrecht = round(df.query(f"CPF in @tenure_entrepreneur_odebrecht_cpfs & Ano == {ano} & cnpj in @cnpjs_odebrecht")["Tempo Emprego"].mean(),2)
    
    tenure_bluecollar_odebrecht_cpfs = df.query("CPF in @cpfs_odebrecht")[(df["cbo_2002_director"] == 0) &
                                                                          (df["coorte"] == ano) &
                                                                          (df["Odebrecht"] == 0)].CPF.unique()
    
    tenure_bluecollar_odebrecht = round(df.query(f"CPF in @tenure_bluecollar_odebrecht_cpfs & Ano == {ano} & cnpj in @cnpjs_odebrecht")["Tempo Emprego"].mean(),2)
    
    tenure_entrepreneur_camargo_cpfs = df.query("CPF in @cpfs_camargo")[
                                                                        (df["coorte"] == ano) &
                                                                        (df["cbo_2002_director"] == 1) &
                                                                        (df["Odebrecht"] == 0) &
                                                                        (df["Camargo"] == 0)].CPF.unique()
    
    tenure_entrepreneur_camargo = round(df.query(f"CPF in @tenure_entrepreneur_camargo_cpfs & Ano == {ano} & cnpj in @cnpjs_camargo")["Tempo Emprego"].mean(),2)
    
    tenure_bluecollar_camargo_cpfs = df.query("CPF in @cpfs_camargo")[(df["cbo_2002_director"] == 0) &
                                                                      (df["coorte"] == ano) &
                                                                      (df["Odebrecht"] == 0)].CPF.unique()
    
    tenure_bluecollar_camargo = round(df.query(f"CPF in @tenure_bluecollar_camargo_cpfs & Ano == {ano} & cnpj in @cnpjs_camargo")["Tempo Emprego"].mean(),2)
    
    data = {'Stats_{}'.format(ano): ['N', '%', 'Schooling(median)', 'Age(mean)', 'Tenure(months_mean)'],
        'Entrepreneurs_Odebrecht':[n_entrepreneur_odebrecht, perc_entrepreneur_odebrecht, study_entrepreneur_odebrecht, age_entrepreneur_odebrecht, tenure_entrepreneur_odebrecht],
        'Bluecollar_Odebrecht':[n_bluecollar_odebrecht, perc_bluecollar_odebrecht, study_bluecollar_odebrecht, age_bluecollar_odebrecht, tenure_bluecollar_odebrecht],
        'Entrepreneurs_Camargo':[n_entrepreneur_camargo, perc_entrepreneur_camargo, study_entrepreneur_camargo, age_entrepreneur_camargo, tenure_entrepreneur_camargo],
        'Bluecollar_Camargo':[n_bluecollar_camargo, perc_bluecollar_camargo, study_bluecollar_camargo, age_bluecollar_camargo, tenure_bluecollar_camargo]}
    
    tabela = pd.DataFrame(data)
    tabela.set_index("Stats_{}".format(ano), inplace = True)
    
    
    print(f"A tabela é referente aos funcionários que deixaram a Odebrecht ou a Camargo Corrêa em {ano} e que até o fim de {ano + 2} criaram novas empresas ('entrepreneurs') ou foram contratados por outras empresas fora do grupo econômico de sua empregadora de {ano} ('blue collar')")
    
    print(tabela)

In [138]:
sumstats(2012)

A tabela é referente aos funcionários que deixaram a Odebrecht ou a Camargo Corrêa em 2012 e que até o fim de 2014 criaram novas empresas ('entrepreneurs') ou foram contratados por outras empresas fora do grupo econômico de sua empregadora de 2012 ('blue collar')
                     Entrepreneurs_Odebrecht  Bluecollar_Odebrecht  \
Stats_2012                                                           
N                                     331.00               4769.00   
%                                       6.49                 93.51   
Schooling(median)                       7.00                  7.00   
Age(mean)                              39.32                 33.54   
Tenure(months_mean)                    16.97                 13.66   

                     Entrepreneurs_Camargo  Bluecollar_Camargo  
Stats_2012                                                      
N                                    83.00              773.00  
%                                     9.70        

In [139]:
sumstats(2013)

A tabela é referente aos funcionários que deixaram a Odebrecht ou a Camargo Corrêa em 2013 e que até o fim de 2015 criaram novas empresas ('entrepreneurs') ou foram contratados por outras empresas fora do grupo econômico de sua empregadora de 2013 ('blue collar')
                     Entrepreneurs_Odebrecht  Bluecollar_Odebrecht  \
Stats_2013                                                           
N                                     318.00               5100.00   
%                                       5.87                 94.13   
Schooling(median)                       7.00                  7.00   
Age(mean)                              39.50                 34.31   
Tenure(months_mean)                    15.01                 14.33   

                     Entrepreneurs_Camargo  Bluecollar_Camargo  
Stats_2013                                                      
N                                    61.00              727.00  
%                                     7.74        

In [140]:
sumstats(2014)

A tabela é referente aos funcionários que deixaram a Odebrecht ou a Camargo Corrêa em 2014 e que até o fim de 2016 criaram novas empresas ('entrepreneurs') ou foram contratados por outras empresas fora do grupo econômico de sua empregadora de 2014 ('blue collar')
                     Entrepreneurs_Odebrecht  Bluecollar_Odebrecht  \
Stats_2014                                                           
N                                     252.00               4765.00   
%                                       5.02                 94.98   
Schooling(median)                       7.00                  7.00   
Age(mean)                              38.39                 34.85   
Tenure(months_mean)                    15.44                 13.02   

                     Entrepreneurs_Camargo  Bluecollar_Camargo  
Stats_2014                                                      
N                                   113.00             1339.00  
%                                     7.78        

In [141]:
sumstats(2015)

A tabela é referente aos funcionários que deixaram a Odebrecht ou a Camargo Corrêa em 2015 e que até o fim de 2017 criaram novas empresas ('entrepreneurs') ou foram contratados por outras empresas fora do grupo econômico de sua empregadora de 2015 ('blue collar')
                     Entrepreneurs_Odebrecht  Bluecollar_Odebrecht  \
Stats_2015                                                           
N                                     174.00               4223.00   
%                                       3.96                 96.04   
Schooling(median)                       7.00                  7.00   
Age(mean)                              40.60                 35.38   
Tenure(months_mean)                    18.90                 20.87   

                     Entrepreneurs_Camargo  Bluecollar_Camargo  
Stats_2015                                                      
N                                    53.00              454.00  
%                                    10.45        

In [142]:
sumstats(2016)

A tabela é referente aos funcionários que deixaram a Odebrecht ou a Camargo Corrêa em 2016 e que até o fim de 2018 criaram novas empresas ('entrepreneurs') ou foram contratados por outras empresas fora do grupo econômico de sua empregadora de 2016 ('blue collar')
                     Entrepreneurs_Odebrecht  Bluecollar_Odebrecht  \
Stats_2016                                                           
N                                     161.00               3611.00   
%                                       4.27                 95.73   
Schooling(median)                       7.00                  7.00   
Age(mean)                              39.67                 35.79   
Tenure(months_mean)                    19.00                 24.68   

                     Entrepreneurs_Camargo  Bluecollar_Camargo  
Stats_2016                                                      
N                                    30.00              334.00  
%                                     8.24        